In [75]:
import torch
import numpy
ckpt = './experiment/train/v2/ckpt_foundation_add_val_with_audio_v_10_5/model_epoch_950.pth'
from train import Network
foundation_model = Network().to(torch.device('cuda'))
foundation_model.load_state_dict(torch.load(ckpt))
foundation_model.eval()

Network(
  (vitpartnet): ViTEncoder(
    (audioencode): AudioEncoder(
      (conv2d_1): Conv2d(2, 128, kernel_size=(16, 16), stride=(16, 16), padding=(1, 1))
    )
    (positionnal): PositionalEcoder()
    (addpostional): AddPostional(
      (positional): PositionalEcoder()
      (visualencoder): VisualEncoder(
        (conv2d_1): Conv2d(3, 128, kernel_size=(16, 16), stride=(16, 16))
      )
      (audioencoder): AudioEncoder(
        (conv2d_1): Conv2d(2, 128, kernel_size=(16, 16), stride=(16, 16), padding=(1, 1))
      )
    )
    (vaencode): VAEncode(
      (visual_transformer_encoder): Encoder(
        (layers): ModuleList(
          (0-5): 6 x EncoderLayer(
            (attention): MultiHeadAttention(
              (attention): ScaleDotProductAttention(
                (softmax): Softmax(dim=-1)
              )
              (w_q): Linear(in_features=128, out_features=128, bias=True)
              (w_k): Linear(in_features=128, out_features=128, bias=True)
              (w_v): Lin

In [76]:
import pickle
path = './dataset/pickle/v2/greedy(val_angle)/e9zR4mvMWw7/804341.pkl'
with open(path , 'rb') as f:
    data = pickle.load(f)

In [77]:
from train import AVtrans
visual = []
audios = []
for d in data['obs']:
    visual.append(torch.from_numpy(d['rgb']).float())
    audio = d['spectrogram'][1]
    audios.append(AVtrans.mel_audio(audio))
visual = torch.stack(visual).to('cuda')
audio = torch.stack(audios).to('cuda')


In [78]:
action , angle = foundation_model(audio , visual)

In [79]:
torch.argmax(action , dim = 1)

tensor([1, 1, 1, 3, 1, 3, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 0, 1, 1, 1, 0, 0], device='cuda:0')

In [80]:
numpy.array(data['action_id'][0])

array([1, 1, 1, 2, 1, 1, 1, 2, 1, 3, 1, 1, 3, 1, 2, 1, 3, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 3, 1, 1, 2, 1, 0])

In [81]:
preds = action.argmax(dim=1)  # [batch]

In [87]:
preds == data['action_id'][0]

False

In [90]:
correct = ( action.argmax(dim=1)[:-1] == torch.tensor(data['action_id'][0]).to('cuda')).sum().item()

In [91]:
correct

21

In [92]:
len(data['obs'])

33

In [ ]:
[3, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
[3, 1, 1, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 0]

In [56]:
print(f"predict action is {torch.argmax(action , dim=1)} ,grandtruth is {data['action_id'][0]},\npredict angle is {angle.item()},grandtruth is {data['obs'][2]['angle'][1]}")

RuntimeError: a Tensor with 18 elements cannot be converted to Scalar

In [ ]:
angle

In [ ]:
len(data['obs']) / 3

18